In [1]:
print("TD2 Logistic Regression Assignment")
print("Student Name: Nishat Salma")

TD2 Logistic Regression Assignment
Student Name: Nishat Salma


In [2]:
#CELL 1
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [3]:
data = pd.read_csv("titanic-dataset.csv")  # adjust filename
display(data.head())
display(data.info())
display(data.describe())

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


None

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [4]:
print(data.columns.tolist())

['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']


In [5]:
# --- CLEAN TITANIC DATASET (Final, Safe Version) ---
print("Columns before cleaning:\n", data.columns.tolist())

# Step 1: Fix missing numeric values
for col in ['Age', 'Fare']:
    if col in data.columns:
        data[col] = pd.to_numeric(data[col], errors='coerce')
        data[col] = data[col].fillna(data[col].median())

# Step 2: Handle Embarked safely (supports C/Q/S or numeric or NaN)
if 'Embarked' in data.columns:
    e = data['Embarked'].astype(str).str.strip().str.upper()
    e = e.replace({'C': 0, 'Q': 1, 'S': 2})  # map letters if present
    e = pd.to_numeric(e, errors='coerce')    # convert to numbers safely
    if e.notna().sum() == 0:
        print("⚠️ Embarked column empty — dropping it.")
        data = data.drop(columns=['Embarked'])
    else:
        mode_val = e.dropna().mode().iloc[0]  # guaranteed safe
        data['Embarked'] = e.fillna(mode_val)

# Step 3: Encode Sex
if 'Sex' in data.columns:
    data['Sex'] = data['Sex'].astype(str).str.lower().map({'male': 0, 'female': 1})

# Step 4: Drop columns not needed
cols_to_drop = [c for c in ['PassengerId', 'Name', 'Ticket', 'Cabin'] if c in data.columns]
if cols_to_drop:
    data = data.drop(columns=cols_to_drop)

# Step 5: Confirm
print("\nAfter cleaning:\n")
data.info()
print("\nMissing values per column:\n", data.isnull().sum())
if 'Sex' in data.columns:
    print("Unique Sex values:", data['Sex'].unique())
if 'Embarked' in data.columns:
    print("Unique Embarked values:", data['Embarked'].unique())

Columns before cleaning:
 ['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']

After cleaning:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    int64  
 3   Age       891 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  891 non-null    float64
dtypes: float64(3), int64(5)
memory usage: 55.8 KB

Missing values per column:
 Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64
Unique Sex values: [0 1]
Unique Embarked values: [2. 0. 1.]


In [6]:
# Fix Sex column encoding (handles capitalized strings)
if 'Sex' in data.columns:
    data['Sex'] = data['Sex'].astype(str).str.lower().map({'male': 0, 'female': 1})

print("Unique Sex values after fix:", data['Sex'].unique())
print("Missing values per column:\n", data.isnull().sum())


Unique Sex values after fix: [nan]
Missing values per column:
 Survived      0
Pclass        0
Sex         891
Age           0
SibSp         0
Parch         0
Fare          0
Embarked      0
dtype: int64


In [7]:
# Reload from CSV (resetting any previous modifications)
data = pd.read_csv("titanic-dataset.csv")

# Clean only what's necessary
data['Age'] = data['Age'].fillna(data['Age'].median())
data['Fare'] = data['Fare'].fillna(data['Fare'].median())
data['Embarked'] = data['Embarked'].fillna(data['Embarked'].mode()[0])

# Encode categorical columns properly
data['Sex'] = data['Sex'].map({'male': 0, 'female': 1})
data['Embarked'] = data['Embarked'].map({'C': 0, 'Q': 1, 'S': 2})

# Drop irrelevant columns
data = data.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'])

# Check cleaned data
print("Cleaned Data Summary:")
print(data.info())
print("\nMissing values:\n", data.isnull().sum())
print("\nUnique Sex values:", data['Sex'].unique())
print("Unique Embarked values:", data['Embarked'].unique())

Cleaned Data Summary:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    int64  
 3   Age       891 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  891 non-null    int64  
dtypes: float64(2), int64(6)
memory usage: 55.8 KB
None

Missing values:
 Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

Unique Sex values: [0 1]
Unique Embarked values: [2 0 1]


In [8]:
#CELL 4
# Separate target (y) and predictors (X)
y = data['Survived']
X = data.drop('Survived', axis=1)

print("Target (y) and Predictors (X) created.")
print("X shape:", X.shape)
print("y shape:", y.shape)
print("Student Name: Nishat Salma")

Target (y) and Predictors (X) created.
X shape: (891, 7)
y shape: (891,)
Student Name: Nishat Salma


In [9]:
from sklearn.preprocessing import StandardScaler

# Initialize the scaler
scaler = StandardScaler()

# Fit on X and transform it
X_scaled = scaler.fit_transform(X)

print("Normalization complete.")
print("Example of scaled data:\n", X_scaled[:5])
print("Student Name: Nishat Salma")

Normalization complete.
Example of scaled data:
 [[ 0.82737724 -0.73769513 -0.56573646  0.43279337 -0.47367361 -0.50244517
   0.58595414]
 [-1.56610693  1.35557354  0.66386103  0.43279337 -0.47367361  0.78684529
  -1.9423032 ]
 [ 0.82737724  1.35557354 -0.25833709 -0.4745452  -0.47367361 -0.48885426
   0.58595414]
 [-1.56610693  1.35557354  0.4333115   0.43279337 -0.47367361  0.42073024
   0.58595414]
 [ 0.82737724 -0.73769513  0.4333115  -0.4745452  -0.47367361 -0.48633742
   0.58595414]]
Student Name: Nishat Salma


In [10]:
from sklearn.model_selection import train_test_split

# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.3, random_state=42
)

print("Data split complete.")
print("Training set size:", X_train.shape[0])
print("Testing set size:", X_test.shape[0])
print("Student Name: Nishat Salma")

Data split complete.
Training set size: 623
Testing set size: 268
Student Name: Nishat Salma


In [11]:
from sklearn.linear_model import LogisticRegression

# Initialize model
lr = LogisticRegression(max_iter=1000)

# Train (fit) the model
lr.fit(X_train, y_train)

print("Model training complete.")
print("Student Name: Nishat Salma")

Model training complete.
Student Name: Nishat Salma


In [12]:
# Predict on the test set
y_pred = lr.predict(X_test)

print("Predictions complete.")
print("Example predictions:", y_pred[:10])
print("Student Name: Nishat Salma")

Predictions complete.
Example predictions: [0 0 0 1 1 1 1 0 1 1]
Student Name: Nishat Salma


In [13]:
from sklearn.metrics import accuracy_score

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

print(" Model evaluation complete.")
print(f" Accuracy Score: {accuracy:.4f}")
print("Student Name: Nishat Salma")

 Model evaluation complete.
🎯 Accuracy Score: 0.8134
Student Name: Nishat Salma


In [35]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("logreg", LogisticRegression(max_iter=1000))
])

In [37]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    "logreg__C": [0.01, 0.1, 1, 10, 100],
    "logreg__penalty": ["l2"],  # keep same as TD2 or slides
    "logreg__solver": ["liblinear"]
}

grid = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    cv=5,
    scoring="accuracy",
    n_jobs=-1
)

grid.fit(X_train, y_train)
print("Best parameters:", grid.best_params_)
print("Best CV score:", grid.best_score_)

Best parameters: {'logreg__C': 1, 'logreg__penalty': 'l2', 'logreg__solver': 'liblinear'}
Best CV score: 0.7993161290322581


In [39]:
y_pred = grid.predict(X_test)

In [41]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

print("Test Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

Test Accuracy: 0.8134328358208955

Classification Report:
               precision    recall  f1-score   support

           0       0.82      0.87      0.85       157
           1       0.80      0.73      0.76       111

    accuracy                           0.81       268
   macro avg       0.81      0.80      0.80       268
weighted avg       0.81      0.81      0.81       268


Confusion Matrix:
 [[137  20]
 [ 30  81]]


In [43]:
y_proba = grid.predict_proba(X_test)[:, 1]
from sklearn.metrics import roc_curve, roc_auc_score

fpr, tpr, thresholds = roc_curve(y_test, y_proba)
print("AUC:", roc_auc_score(y_test, y_proba))

plt.figure()
plt.plot(fpr, tpr)
plt.plot([0, 1], [0, 1], linestyle="--")
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.title("ROC Curve")
plt.show()

AUC: 0.8800711539565044


NameError: name 'plt' is not defined